# Lake House Zones with AWS Glue and Athena
3 cách define a Glue Table in a Glue Catalog:
- Use Glue Console to define each field.
- Use SQL to define a table with DDL or create statements.
- Configure a Glue Job to generate a table definition automatically.

## Way 1: Use Glue Console to define each field.
### 1.1. Define a Landing Zone Table
1. Vào **AWS Glue** -> **Data Catalog** -> **Databases** -> **Add database**. Đặt tên là `stedi` -> **Create database**.
2. Ấn vào DB vừa tạo -> **Add Table**. \
   Đặt **Name** là `customer_landing`, chọn Database là `stedi`. \
   Trong Data store, chọn type of source là `S3`. \
   Điền path là `s3://stedi-lake-house-hieu/customer/landing/`. \
   Trong **Data format**, chọn **Classification** là `JSON` -> **Next**.

    <img src="images/aws1.png" width=1500>

5. Trong **Schema**, chọn **Add**. Điền các column name và type sau:
   - customerName, string
   - email, string
   - phone, string
   - birthDay, string
   - serialNumber, string
   - registrationDate, bigint
   - lastUpdateDate, bigint
   - shareWithResearchAsOfDate, bigint
   - shareWithPublicAsOfDate, bigint

    Chọn **Next** -> **Create**.

    <img src="images/aws2.png" width=1500>

6. Vào **Athena** -> **Query editor**. Trước tiên cần set up query result location in S3. Ấn vào **Edit setting** -> Điền `s3://stedi-lake-house-hieu43/athena/` -> **Save**
7. Chạy query:
    ```sql
    SELECT * FROM customer_landing
    ```

    Ấn **Run** và xem KQ.

    <img src="images/aws3.png" width=1500>

### 1.2. Define Accelerometer Landing Zone with Athena
1. Vào CloudShell và chạy
    ```bash
    cd nd027-Data-Engineering-Data-Lakes-AWS-Exercises/project/starter/accelerometer/
    ```

2. Copy tất cả files vào S3
    ```bash
    aws s3 cp ./landing s3://stedi-lake-house-hieu/accelerometer/landing/ --recursive
    ```

    Kiếm tra bucket
    ```bash
    aws s3 ls s3://stedi-lake-house-hieu/accelerometer/landing/
    ```

3. Vào **Athena** -> **Query editor**. Trong **Tables and views**, ấn vào **Create** -> **S3 bucket data**
5. Đặt **Table name** là `accelerometer_landing`. \
   Trong **Dataset**, điền location là `s3://stedi-lake-house-hieu/accelerometer/landing/`.
   Trong **Data format**, chọn **File format** là `JSON`.
   Trong **Column details**, điền các column name và type sau:
    - user, string
    - x, float
    - y, float
    - z, float

    Ấn **Create table**.

    <img src="images/aws4.png" width=600>

6. Query để kiểm tra
    ```sql
    SELECT * FROM accelerometer_landing
    ```

### 1.3. Athena Joins with Glue Catalog
1. Vẫn ở trong **Query Editor.** Tạo table `customer_trusted` từ S3 `s3://stedi-lake-house-hieu/customer/trusted/`. Kiểm tra bằng query:
   ```sql
    SELECT * FROM customer_trusted
    ```
    <img src="images/aws5.png" width=1500>

2. Join `accelerometer_landing` với `customer_trusted`:
    ```sql
    SELECT *
    FROM accelerometer_landing a
    JOIN customer_trusted c
    ON a.user = c.email
    ```

    <img src="images/aws6.png" width=1500>

## Way 2: Use SQL to define a table with DDL or create statements.
Ở cạnh tên DB, ấn vào dấu $\vdots$ -> **Generate table DDL**

<img src="images/aws7.png" width=500>

## Way 3: Configure a Glue Job to generate a table definition automatically
Ta có sensitive data in **accelerometer landing zone**, ta viết một Glue job để filter data and moves compliant records into an **accelerometer trusted zone** for later analysis.
1. Vào **AWS Glue → ETL jobs**. Trong **Create job**, ấn vào **Visual ETL**.
2. Tạo 2 **Data source** là `S3`, Source type là **Data Catalog table**, một source là table `accelerometer_landing`, một source là table `customer_trusted`.
3. Thêm một **Transform** là `Join`, đặt tên là `Customer Privacy Filter`, có **Node parents** là 2 sources trên. **Join type** là Inner join, condition là `user=email`.
4. Thêm một **Transform** là `DropFields`, có Node parents là `Customer Privacy Filter`. Chọn các **DropFields** từ `customerName` đến `ShareWithPublicAsOfDate`, chỉ giữ `user, x, y, z`.
5. Thêm **Data target** là `S3`, **Format** là `JSON`, **Compression Type** là `None`, **Location** là `s3://stedi-lake-house-hieu/accelerometer/trusted/`.

   <img src="images/aws8.png" width=1500>

6. Trong **Job details**, đặt **Name** là `Accelerometer Trusted Zone`, **IAM Role** là `my-glue-service-role`. Còn lại để nguyên, ấn **Save** rồi **Run**.
7. Vào **Athena** -> **Query editor**
8. Tạo table `accelerometer_trusted` từ S3 `s3://stedi-lake-house-hieu/accelerometer/trusted/`.
9. Query để kiểm tra
    ```sql
    SELECT * from acccelerometer_trusted
    ```

    <img src="images/aws9.png" width=1500>
